In [7]:
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [8]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 32
# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [9]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [10]:
# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)



Found 1174 images belonging to 14 classes.
Found 428 images belonging to 14 classes.


In [11]:
# Define the VGG16 base model
def create_InceptionResNetV2_model():
    InceptionResNetV2_model = InceptionResNetV2(
        include_top=False, 
        input_shape=(img_height, img_width, 3)
        )
    InceptionResNetV2_model.trainable = False

    model = Sequential([
        InceptionResNetV2_model,
        GlobalAveragePooling2D(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(14, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())
    return model




In [12]:
#MODEL fit
model = create_InceptionResNetV2_model()
model.fit(train_generator, epochs=15, validation_data=test_generator)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 2, 2, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d_1   (None, 1536)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 64)                98368     
                                                                 
 batch_normalization_407 (Ba  (None, 64)               256       
 tchNormalization)                                               
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                      

In [16]:
#predict
predictions = model.predict(test_generator)



14/14 [==============================] - 2s 142ms/step


In [18]:
print(predictions)
#calculate accuracy
from sklearn.metrics import accuracy_score
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes
accuracy = accuracy_score(y_true, y_pred)
print(accuracy)

[[0.9966562  0.70240897 0.9793976  ... 0.01943718 0.15308352 0.18827246]
 [0.9873552  0.73969376 0.95810765 ... 0.06542902 0.07220162 0.02942387]
 [0.9811354  0.7872501  0.9326008  ... 0.05270735 0.16397616 0.07496586]
 ...
 [0.5558826  0.44052428 0.17427085 ... 0.1508262  0.31709817 0.99996114]
 [0.336446   0.14789732 0.15691516 ... 0.32839343 0.08884767 0.9987191 ]
 [0.15988    0.0115172  0.07373477 ... 0.06187302 0.05615737 0.98324895]]
0.75
